# Simplify the language of an article

## Retrieve Serlo article text for testing

We first define some helper function to retrieve a Serlo article text to be used as a test subject:

In [1]:
import json
import requests

def get_article_from_uuid(uuid_article):
    req = requests.post(
        #"https://de.serlo.org/api/frontend/localhost-graphql-fetch",
        "https://api.serlo-staging.dev/graphql",
        headers = {
            "Content-Type": "application/json",
        },
        json = {
            "query": """
            query {
              uuid(id: %d) {
                ... on Article {
                  currentRevision { 
                    id
                    content
                  }
                }
              }
            }
            """ % uuid_article
        }
    )
    
    return req.json()

def extract_strings_from_object(obj):
    strings = []

    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == 'text':
                strings.append(value)
            else:
                strings.extend(extract_strings_from_object(value))
    elif isinstance(obj, list):
        for item in obj:
            strings.extend(extract_strings_from_object(item))

    return strings

Now we can use that to get the text of an article:

In [2]:
uuid_article = 1795
content = get_article_from_uuid(uuid_article)['data']['uuid']['currentRevision']['content']
article_text = " ".join(extract_strings_from_object(json.loads(content))).replace("  "," ")
display(article_text)

'Die Ableitung einer Funktion f an einer Stelle x gibt die Steigung des Graphen der Funktion an dieser Stelle an. Bezeichnet wird sie zumeist mit f\'(x) . Ist f\'(x_0)>0 , so steigt \xa0der Graph von f an der Stelle x_0 . Ist f\'(x_0)<0 , so fällt der Graph von f an der Stelle x_0 . An den Extremstellen der Funktion und an Terrassenpunkten  gilt: f\'(x_0)=0 . \xa0 Die Ableitung spielt daher eine wichtige Rolle bei der Berechnung der Extrema und bei der Untersuchung der Monotonie einer Funktion. Funktionen, die an jeder Stelle x der Definitionsmenge eine Ableitung besitzen, nennt man differenzierbar . Das Berechnen der Ableitung nennt man\xa0 Differenzieren . Für die Berechnung der Ableitung existieren entsprechende Ableitungsregeln . Definition Die Ableitung an einem Punkt Die Ableitung ist zunächst nur für einen Punkt \\left(x_0\\;\\left|\\;f\\left(x_0\\right)\\right.\\right) \xa0auf dem Graphen einer Funktion f\\left(x\\right) bzw. für eine Stelle x_0 definiert. Sie ist gegeben durch

## Use langchain to change the article text

### OpneAI setup
Use the OpenAI API with GPT-3.5 as the LLM for this first test. We may later try different options like for example Llama-2.

In [3]:
!pip install python-dotenv
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

### Langchain setup

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
chat = ChatOpenAI(temperature=0.0)
template_string = """
Ändere den durch drei Backticks abgegrenzten Text im folgenden Stil: {style}. \
Text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

Change the values of the variables as you wish:

In [6]:
jahrgangsstufe: int = 8
schulart: str = "Hauptschule"

style = """Deutsch \
in einfacher Sprache, so dass selbst lernschwache %d-Klässler der Schulart %s das Thema gut verstehen
""" % (jahrgangsstufe, schulart)

prompt_to_change_article = prompt_template.format_messages(style=style, text=article_text)
print(prompt_to_change_article[0].content)


Ändere den durch drei Backticks abgegrenzten Text im folgenden Stil: Deutsch in einfacher Sprache, so dass selbst lernschwache 8-Klässler der Schulart Hauptschule das Thema gut verstehen
. Text: ```Die Ableitung einer Funktion f an einer Stelle x gibt die Steigung des Graphen der Funktion an dieser Stelle an. Bezeichnet wird sie zumeist mit f'(x) . Ist f'(x_0)>0 , so steigt  der Graph von f an der Stelle x_0 . Ist f'(x_0)<0 , so fällt der Graph von f an der Stelle x_0 . An den Extremstellen der Funktion und an Terrassenpunkten  gilt: f'(x_0)=0 .   Die Ableitung spielt daher eine wichtige Rolle bei der Berechnung der Extrema und bei der Untersuchung der Monotonie einer Funktion. Funktionen, die an jeder Stelle x der Definitionsmenge eine Ableitung besitzen, nennt man differenzierbar . Das Berechnen der Ableitung nennt man  Differenzieren . Für die Berechnung der Ableitung existieren entsprechende Ableitungsregeln . Definition Die Ableitung an einem Punkt Die Ableitung ist zunächst nur 

In [7]:
changed_article = chat(prompt_to_change_article)
print(changed_article.content)

Die Ableitung einer Funktion f an einer Stelle x zeigt an, wie steil der Graph der Funktion an dieser Stelle ist. Man schreibt meistens f'(x) dafür. Wenn f'(x_0)>0 ist, steigt der Graph von f an der Stelle x_0. Wenn f'(x_0)<0 ist, fällt der Graph von f an der Stelle x_0. An den Extremstellen der Funktion und an Terrassenpunkten gilt: f'(x_0)=0. Die Ableitung ist wichtig, um die Extrema und die Monotonie einer Funktion zu berechnen. Funktionen, bei denen man für jede Stelle x in der Definitionsmenge eine Ableitung berechnen kann, nennt man differenzierbar. Das Berechnen der Ableitung nennt man Differenzieren. Es gibt Regeln, um die Ableitung zu berechnen. 

Die Ableitung ist zunächst nur für einen Punkt (x_0|f(x_0)) auf dem Graphen einer Funktion f(x) oder für eine Stelle x_0 definiert. Man kann sie berechnen, indem man den Differenzenquotienten verwendet. Der Differenzenquotient gibt die Steigung der Sekante durch den Punkt (x_0|f(x_0)) und einen anderen Punkt auf dem Funktionsgraphen 